## 模仿官方的调用测试

In [2]:
l1 = [1,2,3]
l2 = ["a", "b", "c"]
for i in zip(l1, l2):
    print(i)

(1, 'a')
(2, 'b')
(3, 'c')


In [3]:
!nvidia-smi

Mon Mar  3 14:30:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.07                 Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8              3W /  175W |     512MiB /  16376MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

e:\repos\YOLOv8Core


In [1]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103  Python-3.12.4 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Setup complete  (32 CPUs, 31.6 GB RAM, 53.4/512.0 GB disk)


In [6]:
from ultralytics import YOLO;
from IPython.display import display, Image;

In [4]:
!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source='./dog.jpeg' save=False

In [13]:
model = YOLO(f"{HOME}/weights/yolov8m.pt")
results = model.predict(source='./dog.jpeg', conf=0.25, save=False)


image 1/1 e:\repos\YOLOv8Core\dog.jpeg: 640x384 1 person, 1 car, 1 dog, 1 backpack, 36.1ms
Speed: 3.2ms preprocess, 36.1ms inference, 5.4ms postprocess per image at shape (1, 3, 640, 384)


## 检查模型结构+输出结果结构

In [12]:
results[0].__dict__.keys()
# dir(results[0])

dict_keys(['orig_img', 'orig_shape', 'boxes', 'masks', 'probs', 'keypoints', 'obb', 'speed', 'names', 'path', 'save_dir', '_keys'])

In [23]:
results[0].orig_shape,results[0].boxes\
# results[0].names
results[0].boxes.xyxy.cpu().numpy()

array([[     68.527,      249.58,      644.23,      929.14],
       [          0,      354.16,      637.18,      1275.4],
       [     627.43,      733.73,      695.97,      787.49],
       [          0,      675.53,      439.95,      1279.5]], dtype=float32)

In [21]:
[results[0].names[cls] for cls in results[0].boxes.cls.cpu().numpy()]

['dog', 'person', 'car', 'backpack']

In [10]:
import cv2;
import numpy as np;
import matplotlib.pyplot as plt;
import ultralytics;
import torchvision;
import torch;

device = 'cuda' if torch.cuda.is_available() else 'cpu';
model = torch.load('./weights/yolov8m.pt', map_location=device);
model['model'],model.keys()

(DetectionModel(
   (model): Sequential(
     (0): Conv(
       (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (1): Conv(
       (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (2): C2f(
       (cv1): Conv(
         (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
         (act): SiLU(inplace=True)
       )
       (cv2): Conv(
         (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
         (act): SiLU(inp

In [8]:
model['model'].info()

YOLOv8m summary: 295 layers, 25,902,640 parameters, 0 gradients


(295, 25902640, 0, 0.0)

## 尝试构建基于ultralytics的视频检测框架

In [21]:
import cv2;
import numpy as np;
import matplotlib.pyplot as plt;
import ultralytics;

ultralytics.checks()

from ultralytics import YOLO

model = YOLO('./weights/yolov8m.pt');
results = model.predict(source='crossing.jpg', show=True, save=False);

cv2.waitKey(0);
cv2.destroyAllWindows();

Ultralytics YOLOv8.2.103  Python-3.12.4 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Setup complete  (32 CPUs, 31.6 GB RAM, 53.3/512.0 GB disk)

image 1/1 e:\repos\YOLOv8Core\crossing.jpg: 384x640 5 persons, 3 cars, 2 motorcycles, 1 truck, 11.7ms
Speed: 1.7ms preprocess, 11.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


In [12]:
im = cv2.imread('deer&dear.jpg');
results = model.predict(im,conf=0.25, show=True);
cv2.waitKey(0);
cv2.destroyAllWindows();
cv2.imwrite('./outputs/deer&dear.jpg',results[0].orig_img);


0: 640x448 1 person, 2 sheeps, 1 cow, 1 backpack, 1 handbag, 77.7ms
Speed: 4.2ms preprocess, 77.7ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 448)


In [9]:
import cv2;
import numpy as np;
import matplotlib.pyplot as plt;
import ultralytics;

ultralytics.checks()

from ultralytics import YOLO

vdo = cv2.VideoCapture("./videos/test.mp4");
model = YOLO("./weights/yolov8m.pt");
count = 0;
fps = 60



while True:
    ret, ori_img = vdo.read();
    
    if not ret:
        break;
    
    result = model.predict(ori_img,conf=0.25, show=True);
    boxes = result[0].boxes
    
    
    key = cv2.waitKey(int(1000/fps));
    
    if key == ord('q'):
        break;
cv2.destroyAllWindows();
    
    # 获取图片
    # img = result.imgs
    # plt.imshow(img);
    # plt.axis("off");
    # plt.show();
    
    # 获取不同的类型与方框位置
    # boxes

Ultralytics YOLOv8.2.103  Python-3.12.4 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16376MiB)
Setup complete  (32 CPUs, 31.6 GB RAM, 53.3/512.0 GB disk)

0: 384x640 19 persons, 4 bicycles, 2 benchs, 1 handbag, 1 potted plant, 497.2ms
Speed: 20.3ms preprocess, 497.2ms inference, 271.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 3 bicycles, 1 bench, 1 handbag, 1 potted plant, 59.2ms
Speed: 12.9ms preprocess, 59.2ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 3 bicycles, 1 bench, 1 handbag, 1 potted plant, 28.8ms
Speed: 16.7ms preprocess, 28.8ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 3 bicycles, 1 bench, 1 handbag, 1 potted plant, 67.0ms
Speed: 11.6ms preprocess, 67.0ms inference, 9.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 3 bicycles, 1 handbag, 1 potted plant, 32.5ms
Speed: 6.6ms preprocess, 32.5ms inference, 4.

In [35]:
print(boxes.cls)

tensor([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 26.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1., 58.,  0.,  0.,  0., 13., 24.,  0.], device='cuda:0')


In [1]:
model['model']

NameError: name 'model' is not defined